# Codes for running large scale Wind Data Downloads

The script uses shapely points querried from pywkt site lookup written to csv in spatial intersect. 

In [1]:
# Select state, year, and attributes for which to download data
state = "Iowa"
interval = 5 # set wind resolution to 5 min (default is sample every 60 mins)
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014] # available years
attributes = ['windspeed_10m', 'windspeed_100m', 'temperature_10m', 'temperature_100m', 'winddirection_100m']

In [2]:
# Package Imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from io import StringIO
import sys
import time
import csv
import logging
import os
from os import path
from shapely import wkb, wkt
from shapely.geometry import Point, shape, Polygon
import traceback
import pywtk
from pywtk.site_lookup import get_3tiersites_from_wkt
import fiona

# Imports for timing download
from tqdm import tqdm
from tqdm.notebook import trange

In [3]:
# read csvs
wkt_locations = pd.read_csv("US_wind_locations_3.csv")
corn = pd.read_csv("~/Documents/Schoolwork/Junior Fall/EPS 168/project/data/data_yield/Maize_1999_2019_NASS.csv")

In [4]:
# Clean Data
corn.rename(columns={'State ANSI': 'STATEFP', 'County ANSI':'COUNTYFP'}, inplace=True) # match columns
corn_clean = corn.dropna(subset=['STATEFP','COUNTYFP']).copy()

# Add modified FIPS column to yield and wkt data 
corn_clean['ST_CT'] = [(str(elt[0])+"_"+str(int(elt[1]))) for elt in \
                       zip(np.array(corn_clean['STATEFP'].values), np.array(corn_clean['COUNTYFP'].values))]
wkt_locations['ST_CT'] = [(str(elt[0])+"_"+str(elt[1])) for elt in \
                       zip(np.array(wkt_locations['STATEFP'].values), np.array(wkt_locations['COUNTYFP'].values))]

# extract wkt locations which contain a FIPS matching one in the corn dataset
to_download = wkt_locations.loc[wkt_locations['ST_CT'].isin(set(corn_clean['ST_CT'].values))]
to_download.head()

,Unnamed: 0,NAME,STATE,STATEFP,COUNTYFP,NAMELSAD,POINT,SITE,ST_CT
0,0,Cuming,Nebraska,31,39,Cuming County,POINT (-96.985748 41.851967),66913,31_39
5,5,Lancaster,Nebraska,31,109,Lancaster County,POINT (-96.723145 40.7766),54135,31_109
6,6,Lancaster,Nebraska,31,109,Lancaster County,POINT (-96.575378 40.703255),53444,31_109
7,7,Lancaster,Nebraska,31,109,Lancaster County,POINT (-96.7229 40.75803),53954,31_109
8,8,Nuckolls,Nebraska,31,129,Nuckolls County,POINT (-98.216003 40.032036),48472,31_129


In [5]:
# Select sites from dataframe which match state for download
selected_download = to_download.loc[to_download['STATE']== state]
print("The following will implement the download of {} wind data sites.".format(len(selected_download)))
selected_download.head()

The following will implement the download of 267 wind data sites.


,Unnamed: 0,NAME,STATE,STATEFP,COUNTYFP,NAMELSAD,POINT,SITE,ST_CT
266,266,Decatur,Iowa,19,53,Decatur County,POINT (-93.77731300000001 40.8857),55562,19_53
267,267,Decatur,Iowa,19,53,Decatur County,POINT (-93.557129 40.862282),55352,19_53
368,368,Cedar,Iowa,19,31,Cedar County,POINT (-90.993286 41.851456),68641,19_31
369,369,Cedar,Iowa,19,31,Cedar County,POINT (-91.016632 41.87117),68898,19_31
370,370,Cedar,Iowa,19,31,Cedar County,POINT (-90.96993999999999 41.831738),68365,19_31


In [6]:
# Functions for downloading data
def point_download(point, interval, years):
    url = 'https://developer.nrel.gov/api/wind-toolkit/v2/wind/wtk-download.csv?'
    df = 'tmp'
    for yr in range(0,len(years)):
        time.sleep(2)
        year = years[yr]
        params = { 'api_key' : '037P8C9W1gMsO4cNFDFvUYvTDnIhMlA3NSVhXdCC', 'wkt' : point,
                   'names' : [year], 'email' : 'julians3.1415@gmail.com', 'interval': interval}
        response = requests.get(url, params)
        raw_data = StringIO(response.text)
        df_yr = pd.read_csv(raw_data, sep = ",", header=1)
        if type(df)==str:
            df = df_yr
        else:
            df = pd.concat([df, df_yr], axis=0)
    return df

In [11]:
%%time
# Iterate through dataframe for download (ok to do so because time is within loop)
for index in tqdm(range(73+69+84+18,len(selected_download))):
    row = selected_download.iloc[index]
    state, county, site = row['STATE'], row['NAME'], row['SITE'] # parameters for naming
    point = row['POINT'] # extract point for download
    df = point_download(point, interval, years)
    df.to_csv("~/Downloads/Direcho/{}/{}_{}_{}_{}.csv".format(state, site, state, county, interval)) # save to csv

 17%|█▋        | 4/23 [12:45<57:51, 182.70s/it]/Users/julianschmitt/opt/anaconda3/envs/pywtk/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
100%|██████████| 23/23 [1:17:52<00:00, 203.17s/it]

CPU times: user 6min 29s, sys: 42.1 s, total: 7min 11s
Wall time: 1h 17min 53s


## Meeting 11/6
How do we optimiize a damage function so that we get the yield outcome as close as possible. MLR - convert every windspeed into a unit of damage. Look at log yield (fraction of yield). Different yields in different places - accounts for this. As windspeed decreases above 15m a second, take the excess and multiply by $\beta$ coefficient. Increase 1%/(m/s) per event. Transform windspeed into damages and then sum damages. This is now an accumulated damage. One outcome and one yield. Adjust the rate at which the damage is occuring. 